In [2]:
library(caret)
library(xgboost)
#library(MlBayesOpt)
library(Matrix)
library(rBayesianOptimization)
#https://gitlab.com/avinashbarnwal/elokaggle/blob/master/code/304_hyperparameter_optuna.ipynb
#https://gitlab.com/avinashbarnwal/elokaggle/blob/master/code/302_LGBM_BO_hyperpara.ipynb
#https://cran.r-project.org/web/packages/MlBayesOpt/vignettes/MlBayesOpt.html

In [3]:
data_import =function(dataname){
  filename = paste('https://raw.githubusercontent.com/avinashbarnwal/GSOC-2019/master/AFT/test/data/neuroblastoma-data-master/data/',dataname,'/',sep="")
  inputFileName = paste(filename,'inputs.csv',sep="")
  labelFileName = paste(filename,'outputs.csv',sep="")
  foldsFileName = paste(filename,'cv/equal_labels/folds.csv',sep="")
  inputs        = read.table(inputFileName,sep=",",header=T,stringsAsFactors = F,row.names=1)
  labels        = read.table(labelFileName,sep=",",header=T,stringsAsFactors = F,row.names=1)
  folds         = read.table(foldsFileName,sep=",",header=T,stringsAsFactors = F,row.names=1)
  res           = list()
  res$inputs    = inputs
  res$labels    = labels
  res$folds     = folds
  return(res)
}

In [4]:
data_massage = function(inputs,labels){
    rownamesInput = rownames(inputs)
    inputs        = do.call(data.frame,lapply(inputs, function(x) replace(x, is.infinite(x),NA)))
    naColumns     = colnames(inputs)[colSums(is.na(inputs))>0]
    noVarCol      = getNonVarCols(inputs)
    removeCols    = c(naColumns,noVarCol)
    inputs        = inputs[ , !(colnames(inputs) %in% removeCols)]
    rownames(inputs) = rownamesInput
    labels$min.log.lambda = unlist(lapply(labels$min.log.lambda,exp))
    labels$max.log.lambda = unlist(lapply(labels$max.log.lambda,exp))
    res        = list()
    res$inputs = inputs
    res$labels = labels
    return(res)
}

In [5]:
getXY<-function(foldNo,folds,inputs,labels){
    test.id       = rownames(subset(folds,fold==foldNo))
    train.id      = rownames(subset(folds,fold!=foldNo))
    X             = subset(inputs,rownames(inputs) %in% train.id)
    X             = as.matrix(X)
    X.val         = subset(inputs,rownames(inputs) %in% test.id)
    X.val         = as.matrix(X.val)
    y.label       = subset(labels,rownames(labels) %in% train.id)
    y.label.test  = subset(labels,rownames(labels) %in% test.id)
    y.lower       = as.matrix(y.label$min.log.lambda)
    y.upper       = as.matrix(y.label$max.log.lambda)
    y.lower.val   = as.matrix(y.label.test$min.log.lambda)
    y.upper.val   = as.matrix(y.label.test$max.log.lambda)
    res           = list()
    res$X         = X
    res$X.val     = X.val
    res$y.lower      = y.lower
    res$y.lower.val  = y.lower.val
    res$y.upper      = y.upper
    res$y.upper.val  = y.upper.val
    return(res)
}

In [6]:
getNonVarCols<-function(data){
    var_columns    = apply(inputs,2,var)
    resCol         = names(var_columns[var_columns==0.0])
    return(resCol)
}

In [7]:
# Set Parameters
dataNameRange       = c('ATAC_JV_adipose','CTCF_TDH_ENCODE','H3K27ac-H3K4me3_TDHAM_BP','H3K27ac_TDH_some','H3K36me3_AM_immune')
sigma_range         = c(1,2,5,10,100)
distribution_range  = c('normal','logistic','extreme')
learning_rate       = 0.1
num_round           = 200

In [9]:
getaccuracy=function(pred,y_lower,y_higher){
    res = (pred>=y_lower & pred<=y_higher)
    return(res)
}

In [10]:
getParam = function(sigma,distribution,learning_rate){
  eval_metric = paste("aft-nloglik@",distribution,",",sigma,sep="") 
  param       = list(learning_rate=learning_rate, aft_noise_distribution=distribution, 
                    nthread = 4, verbosity=0, aft_sigma= sigma,
                    eval_metric  = eval_metric,
                    objective  = "aft:survival")
  return(param)
}

In [11]:
trainModel = function(foldNo,X,X_val,y_lower,y_lower_val,y_upper,y_upper_val,param,num_round){
  
  dtrain = xgb.DMatrix(X)
  setinfo(dtrain,'label_lower_bound', y_lower)
  setinfo(dtrain,'label_upper_bound', y_upper)
  
  dtest = xgb.DMatrix(X_val)
  setinfo(dtest,'label_lower_bound', y_lower_val)
  setinfo(dtest,'label_upper_bound', y_upper_val)
  
  watchlist = list(eval = dtest, train = dtrain)
  bst       = xgb.train(param, dtrain, num_round, watchlist,verbose = 0)

  return(bst)
}

In [12]:
xgb_cv_grid = function(max_depth, min_child_weight, subsample, 
                        alpha, lambda, nrounds, eta, sigma, distribution,dtrain) {
eval_metric = paste("aft-nloglik@",distribution,",",sigma,sep="") 
cv = xgb.cv(params = list(booster = "gbtree",
                              max_depth = max_depth,min_child_weight = min_child_weight,
                              subsample = subsample,alpha = alpha,
                              lambda = lambda,nrounds = nrounds,
                              learning_rate = learning_rate,
                              aft_sigma = sigma,aft_noise_distribution = distribution,
                              objective = "aft:survival",
                              nthread = 4,
                              eval_metric = eval_metric),
                              data = dtrain,
                              nfold = 5, prediction = TRUE, showsd = TRUE,
                              early_stopping_rounds = 10, maximize = TRUE, verbose = 0)
                              #Score = cv$evaluation_log$test_auc_mean[cv$best_iteration]
}

In [13]:
xgb_grid         = expand.grid(
                                nrounds = 10000,
                                eta     = c(0.001,0.01,0.1,0.5,1),
                                max_depth = c(2, 4, 6, 8, 10),
                                min_child_weight =  c(0.1,10,100), 
                                subsample        =  0.7,
                                alpha            =  c(0.0,0.1,10,100),
                                lambda           =  c(0.0001,0.1,10,100),
                                sigma            =  c(1,2,5,10,100),
                                distribution     =  c('normal','logistic','extreme')
                             )

In [14]:
nParam = dim(xgb_grid)[0]

In [15]:
for(i in 1:1){
    res                 = data_import(dataNameRange[1])
    inputs              = res$inputs
    labels              = res$labels
    folds               = res$folds
    resDataMassage      = data_massage(inputs,labels)
    inputs              = resDataMassage$inputs
    labels              = resDataMassage$labels
    fold_iter           = unique(folds$fold)
    accuracy_fold       = numeric(length(fold_iter))
    res                 = getXY(fold_iter[i],folds,inputs,labels)
    X                   = res$X
    X.val               = res$X.val
    y.lower             = res$y.lower
    y.lower.val         = res$y.lower.val
    y.upper             = res$y.upper
    y.upper.val         = res$y.upper.val
    train.folds         = cut(seq(1,nrow(X)),breaks=5,labels=FALSE)
    res                 = list()
    
    dtrain = xgb.DMatrix(X)
    setinfo(dtrain,'label_lower_bound', y.lower)
    setinfo(dtrain,'label_upper_bound', y.upper)
    
    for(i in 1:1){
        nrounds   = xgb_grid[i,'nrounds']
        eta       = xgb_grid[i,'eta']
        max_depth = xgb_grid[i,'max_depth']
        min_child_weight =  xgb_grid[i,'min_child_weight']
        subsample        =  xgb_grid[i,'subsample']
        alpha            =  xgb_grid[i,'alpha']
        lambda           =  xgb_grid[i,'lambda']
        sigma            =  xgb_grid[i,'sigma']
        distribution     =  xgb_grid[i,'distribution']
        xgb_cv_grid(nrounds,eta,max_depth,min_child_weight,subsample,alpha,lambda,sigma,distribution,dtrain)
}
}

ERROR: Error in xgb.cv(params = list(booster = "gbtree", max_depth = max_depth, : Labels must be provided for CV either through xgb.DMatrix, or through 'label=' when 'data' is matrix
